In [1]:
import gurobipy as gb
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### Functions

In [2]:
# Pickup 
def optimize_shortest_path(tij_matrix, initial_points, delta_s):
    nodes = tij_matrix.columns.tolist()  # List of nodes
    n_nodes = len(nodes)
    tij_values = tij_matrix.values  # Travel times as NumPy array

    # Create a Gurobi model
    model = Model("ShortestPathOptimization")
    model.setParam('OutputFlag', 0)

    # Decision variables: x[i, j] = 1 if path includes edge (i, j), 0 otherwise
    x = model.addVars(n_nodes, n_nodes, vtype=GRB.BINARY, name="x")

    # Objective: Minimize total travel time
    model.setObjective(
        quicksum(round(tij_values[i, j], 1) * x[i, j] for i in range(n_nodes) for j in range(n_nodes)), 
        GRB.MINIMIZE
    )

    # Constraints
    # 1. Flow conservation for initial points I
    for i in [nodes.index(p) for p in initial_points]:
        model.addConstr(quicksum(x[i, j] for j in range(n_nodes)) - quicksum(x[j, i] for j in range(n_nodes)) >= 0)

    # 2. Single flow through initial points I
    for i in [nodes.index(p) for p in initial_points]:
        model.addConstr(quicksum(x[i, j] for j in range(n_nodes)) - quicksum(x[j, i] for j in range(n_nodes)) <= 1)

    # 3. Exactly one initial point contributes to the flow
    model.addConstr(
        quicksum(
            quicksum(x[i, j] for j in range(n_nodes)) - quicksum(x[j, i] for j in range(n_nodes))
            for i in [nodes.index(p) for p in initial_points]
        ) == 1
    )

    # 4. Flow conservation for other nodes (not in I or Δ_S)
    for i in range(n_nodes):
        if nodes[i] not in initial_points + delta_s:
            model.addConstr(
                quicksum(x[i, j] for j in range(n_nodes)) - quicksum(x[j, i] for j in range(n_nodes)) == 0
            )

    # 5. Flow enters the destination node m ∈ Δ_S
    for i in [nodes.index(p) for p in delta_s]:
        if nodes[i] == delta_s[0]:
            model.addConstr(
                quicksum(x[i, j] for j in range(n_nodes)) - quicksum(x[j, i] for j in range(n_nodes)) == -1
            )

    # Solve the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Extract the path and total travel time
        path = []
        for i in range(n_nodes):
            for j in range(n_nodes):
                if x[i, j].x > 0.5:  # Edge (i, j) is part of the path
                    path.append((nodes[i], nodes[j]))
        
        total_time = round(model.objVal, 1)
        return {
            "path": path,
            "total_time": total_time,
        }
    else:
        raise Exception("No feasible solution found")

# Delivery baseline
def shortest_path_tij(tij_matrix, start, end):
    nodes = tij_matrix.columns  # Extract node labels
    n_nodes = len(nodes)
    tij_values = tij_matrix.values  # Convert to NumPy array for optimization

    model = Model("ShortestPath")
    model.setParam('OutputFlag', 0)
    x = model.addVars(n_nodes, n_nodes, vtype=GRB.BINARY, name="x")
    model.setObjective(
        quicksum(round(tij_values[i, j], 1) * x[i, j] for i in range(n_nodes) for j in range(n_nodes)), 
        GRB.MINIMIZE
    )

    # Constraints
    for i in range(n_nodes):
        # Flow balance: Outgoing - Incoming flow = 1 for start, -1 for end, and 0 for other nodes
        flow_balance = quicksum(x[i, j] for j in range(n_nodes)) - quicksum(x[j, i] for j in range(n_nodes))
        if nodes[i] == start:
            model.addConstr(flow_balance == 1, name=f"flow_start_{nodes[i]}")
        elif nodes[i] == end:
            model.addConstr(flow_balance == -1, name=f"flow_end_{nodes[i]}")
        else:
            model.addConstr(flow_balance == 0, name=f"flow_balance_{nodes[i]}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Extract the shortest path
        path = []
        current_node = nodes.get_loc(start)
        while current_node != nodes.get_loc(end):
            path.append(nodes[current_node])
            for j in range(n_nodes):
                if x[current_node, j].x > 0.5:  # Edge (current_node, j) is part of the path
                    current_node = j
                    break
        path.append(end)
        
        # Total travel time
        time = round(model.objVal, 1)
        return path, time
    else:
        raise Exception("No feasible solution found")


# Delivery comparison
def compare_routes(tij_matrix, source, drop_points):
    if len(drop_points) == 1:
        b = drop_points[0]
        u_a_b, time_a_b = shortest_path_tij(tij_matrix, source, b)
        return {
            "shortest_path": u_a_b,
            "total_time": time_a_b,
            "route_description": f"u({source}, {b})",
        }
    elif len(drop_points) == 2:
        b, c = drop_points

        # Compute u(a, b), u(b, c), and u(a, c), u(c, b)
        u_a_b, time_a_b = shortest_path_tij(tij_matrix, source, b)
        u_b_c, time_b_c = shortest_path_tij(tij_matrix, b, c)
        u_a_c, time_a_c = shortest_path_tij(tij_matrix, source, c)
        u_c_b, time_c_b = shortest_path_tij(tij_matrix, c, b)

        # Compute total times for both routes
        route_1_time = time_a_b + time_b_c  # u(a, b) + u(b, c)
        route_2_time = time_a_c + time_c_b  # u(a, c) + u(c, b)

        # Compare and select the shortest route
        if route_1_time < route_2_time:
            shortest_path = u_a_b + u_b_c[1:]  # Combine paths (remove duplicate node)
            total_time = route_1_time
            route_description = f"u({source}, {b}) + u({b}, {c})"
        else:
            shortest_path = u_a_c + u_c_b[1:]  # Combine paths (remove duplicate node)
            total_time = route_2_time
            route_description = f"u({source}, {c}) + u({c}, {b})"

        # Return the result
        return {
            "shortest_path": shortest_path,
            "total_time": total_time,
            "route_description": route_description,
        }
    else:
        raise ValueError("drop_points must contain 1 or 2 nodes only.")

### T matrix

In [3]:
tij_base_data = pd.read_excel("data.xlsx", sheet_name="Tijj")
tij_matrix = tij_base_data.set_index('(Minutes)').apply(pd.to_numeric)


### Example

In [4]:
initial_points = ['D1', 'D2', 'D3']
delta_s = ['S3']
delta_d = ['D26', 'D36']
source_node = delta_s[0]

result_p = optimize_shortest_path(tij_matrix, initial_points, delta_s)
result_d = compare_routes(tij_matrix, source_node, delta_d)

# Results
print(f"Shortest Pickup Path: {result_p['path']}")
print(f"Pickup Travel Time: {result_p['total_time']} minutes")
print(f"\nShortest Delivery Path: {result_d['shortest_path']}")
print(f"Delivery Travel Time: {result_d['total_time']} minutes")
print(f"Delivery Route Description: {result_d['route_description']}")
total_travel_time = result_p['total_time']+result_d['total_time']
print(f"\nTotal Travel Time: {total_travel_time} minutes")


Restricted license - for non-production use only - expires 2025-11-24
Shortest Pickup Path: [('D3', 'S3')]
Pickup Travel Time: 6.0 minutes

Shortest Delivery Path: ['S3', 'D26', 'D36']
Delivery Travel Time: 61.2 minutes
Delivery Route Description: u(S3, D26) + u(D26, D36)

Total Travel Time: 67.2 minutes


In [5]:
df = pd.read_excel("data.xlsx", sheet_name="Real-time orders log (100)")

In [6]:
D_strings = [f"D{i}" for i in range(1,37)]
initial_points = D_strings
delta_s = ['S1']
delta_d = ['D20','D25']
source_node = delta_s[0]

result_p = optimize_shortest_path(tij_matrix, initial_points, delta_s)
result_d = compare_routes(tij_matrix, source_node, delta_d)

# Results
print(f"Shortest Pickup Path: {result_p['path']}")
print(f"Pickup Travel Time: {result_p['total_time']} minutes")
print(f"\nShortest Delivery Path: {result_d['shortest_path']}")
print(f"Delivery Travel Time: {result_d['total_time']} minutes")
print(f"Delivery Route Description: {result_d['route_description']}")
total_travel_time = result_p['total_time']+result_d['total_time']
print(f"\nTotal Travel Time: {total_travel_time} minutes")

Shortest Pickup Path: [('D6', 'S1')]
Pickup Travel Time: 3.6 minutes

Shortest Delivery Path: ['S1', 'D25', 'D20']
Delivery Travel Time: 32.0 minutes
Delivery Route Description: u(S1, D25) + u(D25, D20)

Total Travel Time: 35.6 minutes


In [7]:
# Convert the 'Timestamp' column to datetime objects
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S')

# Initialize variables
batches = []
waiting_orders = {}

# Process each order to form batches
for index, row in df.iterrows():
    supply_center = row['S']
    order_time = row['Timestamp']
    demander = row['D']
    
    # Initialize the waiting list for the supply center if it doesn't exist
    if supply_center not in waiting_orders:
        waiting_orders[supply_center] = []
    
    # Check for existing waiting orders
    if waiting_orders[supply_center]:
        last_order_time, demanders = waiting_orders[supply_center][-1]
        time_diff = (order_time - last_order_time).total_seconds() / 60
        
        # If within 10 minutes and less than 2 orders, batch them
        if time_diff <= 10 and len(demanders) < 2:
            demanders.append(demander)
            batch_time = order_time
            batches.append({
                'Timestamp': batch_time.strftime('%H:%M:%S'),
                'Supply Center': supply_center,
                'Demanders': demanders.copy()
            })
            waiting_orders[supply_center] = []  # Reset after batching
        else:
            # Check if the last order has been waiting for more than 10 minutes
            waiting_time = (order_time - last_order_time).total_seconds() / 60
            if waiting_time >= 10:
                batch_time = last_order_time + timedelta(minutes=10)
                batches.append({
                    'Timestamp': batch_time.strftime('%H:%M:%S'),
                    'Supply Center': supply_center,
                    'Demanders': demanders.copy()
                })
                waiting_orders[supply_center] = [(order_time, [demander])]
            else:
                waiting_orders[supply_center].append((order_time, [demander]))
    else:
        # No waiting orders, add the current order to the waiting list
        waiting_orders[supply_center].append((order_time, [demander]))

# Handle any remaining waiting orders after processing all orders
for supply_center, orders in waiting_orders.items():
    for order_time, demanders in orders:
        batch_time = order_time + timedelta(minutes=10)
        batches.append({
            'Timestamp': batch_time.strftime('%H:%M:%S'),
            'Supply Center': supply_center,
            'Demanders': demanders.copy()
        })

# Create the final DataFrame
batch_df = pd.DataFrame(batches)

# Sort the DataFrame by Timestamp
batch_df['Timestamp'] = pd.to_datetime(batch_df['Timestamp'], format='%H:%M:%S')
batch_df = batch_df.sort_values('Timestamp').reset_index(drop=True)
batch_df['Timestamp'] = batch_df['Timestamp'].dt.strftime('%H:%M:%S')

# -----------------------------------------
# Compute travel times and paths per row
# -----------------------------------------
D_strings = [f"D{i}" for i in range(1, 37)]
initial_points = D_strings

# Lists to store results
pickup_times = []
delivery_times = []
total_times = []
pickup_paths = []
delivery_paths = []

for i, row in batch_df.iterrows():
    supply_center = row['Supply Center']
    demanders = row['Demanders']
    
    # Ensure demanders is a list (if needed)
    # from ast import literal_eval
    # if isinstance(demanders, str):
    #     demanders = literal_eval(demanders)
    
    # Remove duplicates
    unique_demanders = list(set(demanders))
    
    delta_s = [supply_center]
    delta_d = unique_demanders
    
    # Compute shortest paths and times for this batch
    result_p = optimize_shortest_path(tij_matrix, initial_points, delta_s)
    result_d = compare_routes(tij_matrix, delta_s[0], delta_d)
    
    pickup_time = result_p['total_time']
    delivery_time = result_d['total_time']
    total_time = pickup_time + delivery_time
    
    # Store results
    pickup_times.append(pickup_time)
    delivery_times.append(delivery_time)
    total_times.append(total_time)
    
    # Store paths
    pickup_paths.append(result_p.get('path', []))  # default to empty list if not found
    delivery_paths.append(result_d.get('shortest_path', []))  # default if not found

# Add the new columns to the DataFrame
batch_df['pickup_time'] = pickup_times
batch_df['delivery_time'] = delivery_times
batch_df['total_time'] = total_times
batch_df['Shortest Pickup Path'] = pickup_paths
batch_df['Shortest Delivery Path'] = delivery_paths

# Display the final DataFrame
print(batch_df)


   Timestamp Supply Center   Demanders  pickup_time  delivery_time  \
0   10:05:42            S2  [D20, D25]          2.5           22.0   
1   10:17:59            S3       [D30]          5.3            7.7   
2   10:18:42            S2  [D17, D19]          2.5            8.8   
3   10:27:03            S1    [D6, D6]          3.6            3.6   
4   10:41:09            S1    [D4, D8]          3.6           33.2   
..       ...           ...         ...          ...            ...   
57  14:44:27            S2  [D21, D26]          2.5           32.2   
58  14:45:00            S4  [D35, D35]          2.4           23.5   
59  14:55:21            S1  [D13, D11]          3.6           22.0   
60  14:59:34            S1    [D5, D3]          3.6           34.9   
61  15:07:10            S2       [D22]          2.5            6.0   

    total_time Shortest Pickup Path Shortest Delivery Path  
0         24.5          [(D19, S2)]         [S2, D20, D25]  
1         13.0          [(D20, S3)]  

In [8]:
batch_df.to_csv("result.csv")